# Rewrite Library

The rewrite library allows you to add rewrite rules and apply them to terms. Currently it only supports rewrites using syntactic unification but it is planned to allow for rewrites modulo an equational theory.

## Setting up

In [1]:
from symcollab.algebra import *
from symcollab.rewrite import *

It's recommended that you set up your constants, variables, and functions ahead of time

In [2]:
f = Function("f", arity = 2)
g = Function("g", 2)
x = Variable("x")
y = Variable("y")
a = Constant("a")
b = Constant("b")
c = Constant("c")

## Rewrite Rules

Consider the following rewrite rule

In [3]:
r = RewriteRule(f(y, g(x, a)), g(y, a))
print(r)

f(y, g(x, a)) → g(y, a)


Now let's apply it to `f(b, g(c, a))`

In [4]:
r.apply(f(b, g(c, a)))

{'': g(b, a)}

Notice how it returns a dictionary. The keys of the dictionary tell you the position in which the rule is applied and the value tells you the outcome.

In [5]:
r2 = RewriteRule(f(a, x), f(a, a))
r2.apply(f(a, f(a, x)))

{'': f(a, a), '2': f(a, f(a, a))}

If a rewrite rule cannot be applied to a term, then nothing is returned

In [8]:
print(r.apply(f(a,b)))

None


## Rewrite Systems

Things get more interesting when we combine multiple rewrite rules into a rewrite system. 

In [9]:
r = RewriteRule(f(x, x), x)
r2 = RewriteRule(f(a, x), b)
rs = RewriteSystem({r, r2})
print(rs)

{
  f(x, x) → x,
  f(a, x) → b,

}


If it is a convergent term rewriting system, we can find the normal form of a term.

In [13]:
normal_form, rules_applied = normal(f(a, f(x, x)), rs)
print("Normal form:", normal_form)
print("Rules applied:", rules_applied)

Normal form: b
Rules applied: [(f(x_1, x_1) → x_1, '2'), (f(a, x) → b, '')]


## Variants

We can use RewriteSystems to find variants of terms.

In [14]:
term = f(a, f(b, b))
vs = Variants(term, rs)

for vi in vs:
    print(vi)

f(a, f(b, b))
f(a, b)
b


Since there can be an infinite amount of variants, they are only computed when requested. One can also check to see if it's finite up to a certain bound.

In [15]:
# Pass a bound of -1 to keep checking forever
is_finite(vs, -1)

True

Finally, we can narrow from one term to another.

In [16]:
print("Rewrite rule from", term, "to", f(a, b), narrow(term, f(a,b), rs, -1))

Rewrite rule from f(a, f(b, b)) to f(a, b) [(f(x_1, x_1) → x_1, '2')]
